# Chapter 11: Training Deep Neural Networks

In [1]:
import os
from functools import partial
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras import (
    datasets,
    callbacks,
    optimizers,
    regularizers,
    layers,
    Sequential,
    activations,
    metrics,
    losses,
    backend,
)


In [2]:
LEARNING_RATE = 5e-4

LOGS_PATH = os.path.join("..", "logs", "chapter_11")
MODEL_PATH = os.path.join("models", "my_cifar_model.h5")

### 8. Practice training a deep neural network on the CIFAR10 image dataset:

*Build a DNN with 20 hidden layers of 100 neurons each (that’s too many, but it’s the point of this exercise). Use He initialization and the ELU activation function.*

In [3]:
backend.clear_session()
tf.random.set_seed(1992)
np.random.seed(1992)

In [4]:
model = Sequential()
model.add(layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(layers.Dense(100, activation="elu", kernel_initializer="he_normal"))

model.add(layers.Dense(10, activation="softmax"))


Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



*Using Nadam optimization and early stopping, train the network on the CIFAR10 dataset. You can load it with keras.datasets.cifar10.load_data(). The dataset is composed of 60,000 32 × 32–pixel color images (50,000 for training, 10,000 for testing) with 10 classes, so you’ll need a softmax output layer with 10 neurons. Remember to search for the right learning rate each time you change the model’s architecture or hyperparameters.*

In [5]:
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
x_train, x_valid = x_train[:40000], x_train[40000:]
y_train, y_valid = y_train[:40000], y_train[40000:]

In [6]:
optimizer = optimizers.Nadam(learning_rate=LEARNING_RATE)

model.compile(
    optimizer=optimizer,
    metrics=["accuracy"],
    loss="sparse_categorical_crossentropy",
)

# Callbacks
early_stop_cb = callbacks.EarlyStopping(patience=5, monitor='val_loss')
tensorboard_cb = callbacks.TensorBoard(log_dir=LOGS_PATH)
model_checkpoint_cb = callbacks.ModelCheckpoint(MODEL_PATH, save_best_only=True)

callbacks_ = [early_stop_cb, tensorboard_cb, model_checkpoint_cb]

In [7]:

model.fit(
    x_train, 
    y_train,
    validation_data=[x_valid, y_valid],
    callbacks=callbacks_,
    epochs=100,
    batch_size=512,
)

model = keras.models.load_model(MODEL_PATH)
model.evaluate(x_test, y_test)

Epoch 1/100
79/79 [==============================] - 10s 94ms/step - loss: 12.5227 - accuracy: 0.1213 - val_loss: 2.8022 - val_accuracy: 0.1451
Epoch 2/100
79/79 [==============================] - 6s 79ms/step - loss: 2.4382 - accuracy: 0.1685 - val_loss: 2.9737 - val_accuracy: 0.1566
Epoch 3/100
79/79 [==============================] - 6s 79ms/step - loss: 2.2385 - accuracy: 0.2008 - val_loss: 2.3230 - val_accuracy: 0.1852
Epoch 4/100
79/79 [==============================] - 6s 78ms/step - loss: 2.1605 - accuracy: 0.2218 - val_loss: 2.3706 - val_accuracy: 0.1853
Epoch 5/100
79/79 [==============================] - 6s 78ms/step - loss: 2.0975 - accuracy: 0.2384 - val_loss: 2.0688 - val_accuracy: 0.2523
Epoch 6/100
79/79 [==============================] - 6s 77ms/step - loss: 2.0594 - accuracy: 0.2512 - val_loss: 2.1007 - val_accuracy: 0.2366
Epoch 7/100
79/79 [==============================] - 6s 79ms/step - loss: 2.0246 - accuracy: 0.2620 - val_loss: 2.1944 - val_accuracy: 0.2022
Epoc

[1.7520089149475098, 0.37040001153945923]

> #### Now try adding Batch Normalization and compare the learning curves: Is it converging faster than before? Does it produce a better model? How does it affect training speed?

In [8]:
backend.clear_session()
tf.random.set_seed(1992)
np.random.seed(1992)

In [9]:
model = Sequential(
    [layers.Flatten(input_shape=[32, 32, 3]), layers.BatchNormalization()]
)

for _ in range(20):
    model.add(layers.Dense(100, kernel_initializer="he_normal"))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation("elu"))

model.add(layers.Dense(10, activation="softmax"))

In [10]:
optimizer = optimizers.Nadam(learning_rate=LEARNING_RATE)

model.compile(
    optimizer=optimizer,
    metrics=["accuracy"],
    loss="sparse_categorical_crossentropy",
)

# Callbacks
early_stop_cb = callbacks.EarlyStopping(patience=5, monitor='val_loss')
tensorboard_cb = callbacks.TensorBoard(log_dir=LOGS_PATH)
model_checkpoint_cb = callbacks.ModelCheckpoint(MODEL_PATH, save_best_only=True)

callbacks_ = [early_stop_cb, tensorboard_cb, model_checkpoint_cb]

In [11]:
model.fit(
    x_train, 
    y_train,
    validation_data=[x_valid, y_valid],
    epochs=100,
    callbacks=callbacks_,
    batch_size=512,
)

model = keras.models.load_model(MODEL_PATH)
model.evaluate(x_test, y_test)

Epoch 1/100
79/79 [==============================] - 30s 261ms/step - loss: 1.8702 - accuracy: 0.3339 - val_loss: 2.6717 - val_accuracy: 0.2384
Epoch 2/100
79/79 [==============================] - 13s 166ms/step - loss: 1.5728 - accuracy: 0.4396 - val_loss: 2.0931 - val_accuracy: 0.3257
Epoch 3/100
79/79 [==============================] - 13s 159ms/step - loss: 1.4674 - accuracy: 0.4809 - val_loss: 1.7412 - val_accuracy: 0.4267
Epoch 4/100
79/79 [==============================] - 12s 158ms/step - loss: 1.3880 - accuracy: 0.5092 - val_loss: 1.7352 - val_accuracy: 0.4111
Epoch 5/100
79/79 [==============================] - 12s 154ms/step - loss: 1.3262 - accuracy: 0.5311 - val_loss: 1.6525 - val_accuracy: 0.4388
Epoch 6/100
79/79 [==============================] - 12s 156ms/step - loss: 1.2709 - accuracy: 0.5513 - val_loss: 1.8323 - val_accuracy: 0.4044
Epoch 7/100
79/79 [==============================] - 12s 155ms/step - loss: 1.2290 - accuracy: 0.5666 - val_loss: 1.6380 - val_accuracy:

[1.6154439449310303, 0.4564000070095062]

> #### Try replacing Batch Normalization with SELU, and make the necessary adjustements to ensure the network self-normalizes (i.e., standardize the input features, use LeCun normal initialization, make sure the DNN contains only a sequence of dense layers, etc.).

In [12]:
backend.clear_session()
tf.random.set_seed(1992)
np.random.seed(1992)

In [14]:
model = Sequential(
    [layers.Flatten(input_shape=[32, 32, 3]), layers.BatchNormalization()]
)

for _ in range(20):
    model.add(layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"))

model.add(layers.Dense(10, activation="softmax"))


> #### Try regularizing the model with alpha dropout. Then, without retraining your model, see if you can achieve better accuracy using MC Dropout.

> #### Retrain your model using 1cycle scheduling and see if it improves training speed and model accuracy.